#### This notebook implements the Value Iteration Algo and applies it to each provided MDP file

##### Including numpy and the defaultdict class

In [21]:
import numpy as np
from collections import defaultdict # (automatically initializes missing keys with a default value)

##### Defining the MDP class and the value iteration algorithm

In [22]:
class MDP: #this class loads an MDP from a txt file and applies the value iteration algo
    def __init__(self, filename):
        self.filename = filename
        self.parse_mdp()

    def parse_mdp(self): #prsing the file
        with open(self.filename, 'r') as f:
            lines = f.readlines()

        self.num_states = int(lines[0].split()[1]) #need to convert to integer for it to work
        self.num_actions = int(lines[1].split()[1])

        end_line = lines[2].split()

        if end_line[1] == '-1':
            self.end_states = []
        else:
            self.end_states = [int(x) for x in end_line[1:]]

        #making 3 nested defaultdict to map state action pairs to next state, rewards and transition probabilities
        self.transitions = defaultdict(lambda: defaultdict(list))
        self.rewards = defaultdict(lambda: defaultdict(list))
        self.probabilities = defaultdict(lambda: defaultdict(list))
        
        #startwith function clutch
        for line in lines[3:]:
            if line.startswith('transition'):
                parts = line.split()
                s, a, s_, r, p = int(parts[1]), int(parts[2]), int(parts[3]), float(parts[4]), float(parts[5])
                self.transitions[s][a].append(s_)
                self.rewards[s][a].append(r)
                self.probabilities[s][a].append(p)
            elif line.startswith('mdptype'):
                self.mdp_type = line.split()[1]
            elif line.startswith('discount'):
                self.discount = float(line.split()[1])

    def get_expected_reward(self, s, a): #function to get immediate expected reward after taking a variable action, also the zip function makes our lives easier
        if a not in self.rewards[s]: return 0.0
        return sum(r * p for r, p in zip(self.rewards[s][a], self.probabilities[s][a]))
    
    def get_expected_value(self, s, a, V): #updating all the value functions by a onestep search
        if a not in self.transitions[s]: return 0.0
        return sum(p * V[s_] for s_, p in zip(self.transitions[s][a], self.probabilities[s][a]))
    
    def value_iteration(self, max_iterations=10000, tol=1e-8):
        V = np.zeros(self.num_states)

        for _ in range(max_iterations):
            V_new = np.zeros_like(V)
            delta = 0.0

            for s in range(self.num_states):
                if s in self.end_states:
                    V_new[s] = 0.0
                else:
                    Qsa = [ #here 'Qsa' is the action value function
                        self.get_expected_reward(s, a) + self.discount * self.get_expected_value(s, a, V)
                        for a in range(self.num_actions)
                    ]
                    V_new[s] = max(Qsa) if Qsa else 0.0 #the value of state value function is the value of the maximum action value function
                delta = max(delta, abs(V_new[s] - V[s]))

            V = V_new

            if delta < tol: #if the change is lesser than a really small number (tol here) then we have the best policy, or atleast we're close to the best policy
                break
            
        return V
    
    def extract_policy(self, V): #getting the policy is finding which action has the highest action value function value so this function is the implementation of that
        pi = np.zeros(self.num_states, dtype=int)
        for s in range(self.num_states):
            if s in self.end_states:
                pi[s] = 0
            else:
                Qsa = [
                    self.get_expected_reward(s, a) + self.discount * self.get_expected_value(s, a, V)
                    for a in range(self.num_actions)
                ]
                pi[s] = np.argmax(Qsa) if Qsa else 0
        return pi

###### Solution for data/continuing-mdp-2-2.txt

In [23]:
mdp1 = MDP('data/continuing-mdp-2-2.txt')
V1 = mdp1.value_iteration()
pi1 = mdp1.extract_policy(V1)
for s in range(mdp1.num_states):
    print(f"{V1[s]:.6f} {pi1[s]}")

5.999299 0
5.918450 0


###### Solution for data/continuing-mdp-10-5.txt

In [24]:
mdp2 = MDP('data/continuing-mdp-10-5.txt')
V2 = mdp2.value_iteration()
pi2 = mdp2.extract_policy(V2)
for s in range(mdp2.num_states):
    print(f"{V2[s]:.6f} {pi2[s]}")

2.234958 3
2.373612 3
2.604046 3
2.647784 1
2.522231 4
2.375252 0
2.684806 2
2.688310 0
2.640809 3
2.572427 1


###### Solution for data/continuing-mdp-50-20.txt

In [25]:
mdp3 = MDP('data/continuing-mdp-50-20.txt')
V3 = mdp3.value_iteration()
pi3 = mdp3.extract_policy(V3)
for s in range(mdp3.num_states):
    print(f"{V3[s]:.6f} {pi3[s]}")

1.065079 7
1.051696 2
0.824259 7
0.601320 14
1.057797 4
0.980877 19
0.983041 18
1.002595 5
0.886921 15
0.837798 8
1.109280 8
0.910305 19
1.155357 7
0.958098 8
0.772395 18
1.218694 16
0.939597 11
0.840961 19
0.934034 2
0.899851 12
1.168103 14
0.985183 19
1.032489 14
1.110618 15
0.779151 0
0.945382 1
1.185461 3
1.083733 18
0.697620 15
1.125198 5
0.556266 1
1.088646 6
0.829482 11
0.884322 6
1.180251 1
0.922217 4
0.916141 11
1.031048 10
1.077761 14
0.900197 19
0.855533 5
1.205419 0
1.056961 4
0.720773 14
1.141582 1
1.110485 4
0.983264 5
1.030596 3
0.779689 1
0.815195 12


###### Solution for data/episodic-mdp-2-2.txt

In [26]:
mdp4 = MDP('data/episodic-mdp-2-2.txt')
V4 = mdp4.value_iteration()
pi4 = mdp4.extract_policy(V4)
for s in range(mdp4.num_states):
    print(f"{V4[s]:.6f} {pi4[s]}")

0.000000 0
1.455816 0


###### Solution for data/episodic-mdp-10-5.txt

In [27]:
mdp5 = MDP('data/episodic-mdp-10-5.txt')
V5 = mdp5.value_iteration()
pi5 = mdp5.extract_policy(V5)
for s in range(mdp5.num_states):
    print(f"{V5[s]:.6f} {pi5[s]}")

0.000000 0
525.256443 3
525.548050 4
500.064054 2
468.517954 1
0.000000 0
522.022358 2
513.605831 2
351.127056 4
524.331008 0


###### Solution for data/episodic-mdp-50-20.txt

In [28]:
mdp6 = MDP('data/episodic-mdp-50-20.txt')
V6 = mdp6.value_iteration()
pi6 = mdp6.extract_policy(V6)
for s in range(mdp6.num_states):
    print(f"{V6[s]:.6f} {pi6[s]}")

7.985542 16
7.837297 9
0.000000 0
7.664216 18
7.830741 15
7.826878 12
7.943427 10
8.261769 4
7.869692 14
8.348371 5
7.711355 11
7.775431 0
7.914741 17
8.006133 16
8.101707 0
8.089338 15
0.000000 0
7.652557 9
8.124858 4
7.843161 15
8.415760 12
7.321340 9
7.627955 2
7.984528 7
7.708910 13
7.777016 10
8.089617 15
5.340502 18
8.238763 19
7.855451 6
7.457378 3
7.829692 0
0.000000 0
7.660101 17
0.000000 0
8.418252 8
7.959227 17
8.097640 0
7.778000 18
7.661630 0
7.991036 3
8.497708 3
7.933301 8
7.623537 19
7.864192 10
7.799442 1
7.948461 7
7.806157 5
7.637896 18
7.745240 18
